In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab03.ipynb")

# Lab 03: Data Cleaning and EDA
## Due Wednesday, June 26th, 11:59 PM PT

To receive credit for a lab, answer all questions correctly and submit before the deadline.

You must submit this assignment to Gradescope by the on-time deadline, **Wednesday, June 26th, 11:59 PM PT**. Please read the syllabus for the grace period policy. No late submissions beyond the grace period will be accepted. While course staff is happy to help you if you encounter difficulties with submission, we may not be able to respond to late-night requests for assistance (TAs need to sleep, after all!). **We strongly encourage you to plan to submit your work to Gradescope several hours before the stated deadline.** This way, you will have ample time to contact staff for submission support. 

### Lab Walk-Through
In addition to the lab notebook, we have also released a prerecorded walk-through video of the lab. We encourage you to reference this video as you work through the lab. Run the cell below to display the videos.
<br>
**Note:** This video was recorded in Fall 2023. **Question 5 is a new addition this summer and will not be included in the video**. Please reach out for help in office hours or on Ed if you have any questions regarding this portion of the lab! 

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("hwRYs5ZRgW4", list = 'PLQCcNQgUcDfqSg049DVFZCQbupMY5Bn5Z', listType = 'playlist')

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** below. (That's a good way to learn your classmates' names.)

**Collaborators**: *list collaborators here*

---
### Debugging Guide
If you run into any technical issues, we highly recommend checking out the [Data 100 Debugging Guide](https://ds100.org/debugging-guide/). In this guide, you can find general questions about Jupyter notebooks / Datahub, Gradescope, and common pandas errors.

---
[`pandas`](https://pandas.pydata.org/) is one of the most widely used `Python` libraries in data science. In this lab, you will review commonly used data-wrangling operations/tools in `pandas`. We continue the content from the previous lab and aim to give you familiarity with:

* Aggregating the data (using `.groupby`),
* Filtering the data (using boolean arrays and `groupby.filter`),
* Pivoting (using `.pivot_table`).

In this lab, you are going to use several `pandas` methods. Reminder from the lecture that you may press `shift+tab` on method parameters to see the documentation for that method. For example, if you were using the `drop` method in `pandas`, you could press `shift+tab` to see what `drop` is expecting.

`pandas` is very similar to the `datascience` library that you saw in Data 8. This [conversion notebook](https://data100.datahub.berkeley.edu/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2FDS-100%2Fsu23-materials&branch=main&urlpath=lab%2Ftree%2Fsu23-materials%2Flec%2Flec02%2Fdata8_translation_examples.ipynb) may serve as a useful guide!

This lab expects that you have watched all three `pandas` lectures. If you have not, this lab will probably take a very long time.

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

### **REVIEW:** `Groupby` and `Groupby` Shorthand

Let's now turn to use `groupby` from lectures 3 and 4.

### Elections

Let's start by reading in the election dataset from the `pandas` lectures.

In [ ]:
# Run this cell to load data from CSV file; no further action is needed.
elections = pd.read_csv("data/elections.csv")
elections.head(5)

As we saw before, we can `groupby` a specific column, e.g., `"Party"` and can print out the resulting sub-DataFrames. The output below can help you get an understanding of what `groupby` is actually doing.

An example is given below for elections since 1980.

In [ ]:
# Run this cell to print sub-DataFrames of a groupby object; no further action is needed.
for n, g in elections[elections["Year"] >= 1980].groupby("Party"):
    print(f"Name: {n}") # By the way, this is an "f string", a relatively new and great feature of Python
    display(g)

Recall that once we've formed groups, we can aggregate each sub-DataFrame (a.k.a. group) into a single row using an aggregation function. For example, if we use `.agg('mean')` on the groups above, we get back a single `DataFrame` where each group has been replaced by a single row. In each column for that aggregate row, the value that appears is the average of all values in that group.

For columns that are non-numeric, e.g., `"Result"`, the `pandas` version we're using (version 2.0.2) will error because we cannot compute the mean of the `Result` column. To remedy this, we add a `numeric_only=True` argument to our function calls so that we only calculate the `mean` for columns that contain numeric values. Alternatively, we can manually select only the numerical columns before calling the `agg` function so the aggregation is only applied to numerical columns.

In [ ]:
elections_after_1980 = elections[elections["Year"] >= 1980]

elections_after_1980.groupby("Party").agg('mean', numeric_only=True)

# alternatively, we can manually select only the numerical columns before calling `agg`
# elections_after_1980.groupby("Party")[['Year', 'Popular vote', '%']].agg('mean')

Equivalently we can use one of the shorthand aggregation functions, e.g. `.mean()`: 

In [ ]:
elections_after_1980.groupby("Party").mean(numeric_only=True)

Note that the index of the `DataFrame` returned by an `groupby.agg` call is no longer a set of numeric indices from $0$ to $N-1$. Instead, we see that the index for the example above is now the `Party`. If we want to restore our `DataFrame` so that `Party` is a column rather than the index, we can use `reset_index`.

In [ ]:
elections_after_1980.groupby("Party").mean(numeric_only=True).reset_index()

**IMPORTANT NOTE:** Notice that the code above consists of chained method calls. This sort of code is very common in `pandas` programming and in data science in general. Such chained method calls can sometimes go many layers deep, so you might consider adding newlines between lines of code for clarity. For example, we could instead write the code above as:

In [ ]:
# pandas method chaining
(
elections.query("Year >= 1980").groupby("Party") 
                               .mean(numeric_only=True)  ## Computes the mean values by party
                               .reset_index()            ## Resets to a numerical index
)

Note that we have surrounded the entire call by a big set of parentheses so that `Python` doesn't complain about the indentation. An alternative is to use the \ symbol to indicate to `Python` that your code continues on to the next line!

In [ ]:
# pandas method chaining (alternative)
elections[elections["Year"] >= 1980].groupby("Party") \
                               .mean(numeric_only=True) \
                               .reset_index()     

**IMPORTANT NOTE:** You should NEVER solve problems like the one above using loops or list comprehensions. This is slow and also misses the entire point of this part of Data 100. 

Before we continue, we'll print out the election dataset again for your convenience. 

In [ ]:
elections.head(5)

<br>

---

### Question 1a
Using `groupby.agg` or one of the shorthand methods (`groupby.min`, `groupby.first`, etc.), create a `Series` object `best_result_percentage_only` that returns a `Series` showing the entire best result for every party, sorted in decreasing order. Your `Series` should include only parties that have earned at least 10% of the vote in some election. Your result should look like this:

<code>
Party
Democratic               61.344703
Republican               60.907806
Democratic-Republican    57.210122
National Union           54.951512
Whig                     53.051213
Liberal Republican       44.071406
National Republican      43.796073
Northern Democratic      29.522311
Progressive              27.457433
American                 21.554001
Independent              18.956298
Southern Democratic      18.138998
American Independent     13.571218
Constitutional Union     12.639283
Free Soil                10.138474
Name: %, dtype: float64
</code>
<br/>

A list of named `groupby.agg` shorthand methods are [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation) (you'll have to scroll down about one page).


In [ ]:
...
best_result_percentage_only

In [ ]:
grader.check("q1a")

<br>

---

### Question 1b  
Repeat Question 1a. However, this time, your result should be a `DataFrame` showing all available information (all columns) rather than only the percentage as a `Series`.

This question is trickier than Question 1a. Make sure to check the `pandas` lecture slides if you're stuck! It's very easy to make a subtle mistake that shows Woodrow Wilson and Howard Taft both winning the 2020 election.

For example, the first 3 rows of your table should be:

|Party | Year | Candidate      | Popular Vote | Result | %         |
|------|------|----------------|--------------|--------|-----------|
|**Democratic**  | 1964 | Lyndon Johnson | 43127041      | win   | 61.344703 |
|**Republican**  | 1972 | Richard Nixon | 47168710      | win   | 60.907806 |
|**Democratic-Republican**  | 1824 | Andrew Jackson | 151271      | loss   | 57.210122 |

Note that the index is `Party`. In other words, don't use `reset_index`.


In [ ]:
...
best_result

In [ ]:
grader.check("q1b")

### **REVIEW:** `DataFrameGroupBy.filter`

Our `DataFrame` contains a number of parties that have never had a successful presidential run. For example, the 2020 elections included candidates from the Libertarian and Green parties, neither of which have elected a president.

In [ ]:
# Run this cell to print the last four rows; no further action is needed.
elections.tail(4)

Suppose we were conducting an analysis trying to focus our attention on parties that had elected a president. 

The most natural approach is to use `groupby.filter` [(docs)](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.filter.html). This is an incredibly powerful but subtle tool for filtering data.

The code below accomplishes the task at hand. It does this by creating a function that returns `True` if and only if a sub-`DataFrame` (a.k.a. group) contains at least one winner. This function, in turn, uses the `pandas` function `any` [(docs)](https://pandas.pydata.org/docs/reference/api/pandas.Series.any.html).

In [ ]:
# Run this cell to keep only the rows of parties that have 
# elected a president; no further action is needed.
def at_least_one_candidate_in_the_frame_has_won(frame):
    """Returns df with rows only kept for parties that have
    won at least one election
    """
    return (frame["Result"] == 'win').any()

winners_only = (
    elections
        .groupby("Party")
        .filter(at_least_one_candidate_in_the_frame_has_won)
)
winners_only.tail(5)

Alternately, we could have used a `lambda` function instead of explicitly defining a named function using `def`. 

In [ ]:
# Run this cell to keep only the rows of parties that have 
# elected a president; no further action is needed.
winners_only = (
    elections
        .groupby("Party")
        .filter(lambda x : (x["Result"] == "win").any())
)
winners_only.tail(5)

<br>

---

### Question 1c

Using `filter`, create a `DataFrame` object `major_party_results_since_1988` that includes all election results starting after 1988 (exclusive) but only shows a row if the Party it belongs to has earned at least 1% of the popular vote in ANY election since 1988.

For example, in 1988, you should not include the `New Alliance` candidate since this party has not earned 1% of the vote since 1988. However, you should include the `Libertarian` candidate from 1988 despite only having 0.47 percent of the vote in 1988, because in 2016 and 2020, the Libertarian candidates Gary Johnson and Jo Jorgensen exceeded 1% of the vote.

For example, the first three rows of the table you generate should look like:

|     |   Year | Candidate         | Party       |   Popular vote | Result   |         % |
|----:|-------:|:------------------|:------------|---------------:|:---------|----------:|
| 139 |   1992 | Andre Marrou      | Libertarian |       290087   | loss     | 0.278516  |
| 140 |   1992 | Bill Clinton      | Democratic  |       44909806 | win      | 43.118485 |
| 142 |   1992 | George H. W. Bush | Republican  |       39104550 | loss     |  37.544784|

*Hint*: The following questions might help you construct your solution. One of the lines should be identical to the `filter` examples shown above.

1) How can we **only** keep rows in the data starting after 1988 (exclusive)?
2) What column should we `groupby` to filter out parties that have earned at least 1% of the popular vote in ANY election since 1988?
3) How can we write an aggregation function that takes a sub-DataFrame and returns whether at least 1% of the vote has been earned in that sub-DataFrame? This may give you a hint about the second question!


In [ ]:
...
major_party_results_since_1988.head()

In [ ]:
grader.check("q1c")

### **REVIEW:** `str`

`pandas` provides special purpose functions for working with specific common data types such as strings and dates, which you will learn about in more detail in Lecture 6. For example, the code below provides the length of every Candidate's name from our `elections` dataset. 

In [ ]:
elections["Candidate"].str.len()

<br>

---

### Question 2

Using `.str.split`, create a new `DataFrame` called `elections_with_first_name` with a new column `First Name` that is equal to the Candidate's first name.

See the `pandas` `str` [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html) for documentation on using `str.split`.

In [ ]:
elections_with_first_name = elections.copy()
...
elections_with_first_name

In [ ]:
grader.check("q2")

<br>

---

## Babynames
Remember the `babynames` dataset from Lab02A? Let's load it in again and explore the data with our newly covered functions! Like last time, we'll only load in data from California. 

Run the following cell: 

In [ ]:
file_path = 'data/namesbystate_ca.txt.gz'
column_labels = ['State', 'Sex', 'Year', 'Name', 'Count']

babynames = pd.read_csv(file_path, names=column_labels)

babynames.head()

The code below creates a table with the frequency of all names from 2022. 

In [ ]:
# Run this cell to create a table with the frequency 
# of all names from 2022; no further action is needed.
babynames_2022 = (
    babynames[babynames['Year'] == 2022]
              .groupby("Name")
              .sum()[["Count"]]
              .reset_index()
)
babynames_2022

<br>

---

### Question 3

Using the [pd.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) function described in the lecture, combine the `babynames_2022` table with the `elections_with_first_name` table you created earlier to form `presidential_candidates_and_name_popularity`. Your resulting `DataFrame` should contain all columns from both of the tables.

In [ ]:
presidential_candidates_and_name_popularity = ...
presidential_candidates_and_name_popularity

In [ ]:
grader.check("q3")

### **REVIEW:** `pandas.pivot_table`

Suppose we want to build a table showing the total number of babies born of each sex in each year. One way is to `groupby` using both columns of interest:

In [ ]:
babynames.groupby(["Year", "Sex"])[["Count"]].agg(sum).head(6)

While this does give us the information we're looking for, a more natural approach is to use pivot tables to represent our data in a more readable format.

In [ ]:
babynames_pivot = babynames.pivot_table(
    index = "Year",     # rows (turned into index)
    columns = "Sex",    # column values
    values = ["Count"], # field(s) to process in each group
    aggfunc = np.sum,   # group operation
)
babynames_pivot.head(6)

We can also include multiple values in our pivot tables 

In [ ]:
babynames_pivot = babynames.pivot_table(
    index = "Year",     # rows (turned into index)
    columns = "Sex",    # column values
    values = ["Count", "Name"],
    aggfunc = np.max,   # group operation
)
babynames_pivot.head(6)

<br>

---

### Question 4

Using `presidential_candidates_and_name_popularity`, create a table, `party_result_popular_vote_pivot`, whose index is the `Party` and whose columns are their `Result`. Each cell should contain the total number of popular votes received. `pandas`' `pivot_table` documentation is [here](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

You may notice that there are `NaN`s in your table from missing data. Replace the `NaN` values with 0. You may find `.pivot_table`'s `fill_value=` argument helpful. Or, you can use `pd.DataFrame.fillna` [(documentation here)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html).

In [ ]:
...
party_result_popular_vote_pivot

In [ ]:
grader.check("q4")

Just for fun: Which historical presidential candidates have names that were the least and most popular in 2022? Note: Here you'll observe a common problem in data science -- one of the least popular names is actually due to the fact that one recent president was so commonly known by his nickname that he appears named as such in the database from which you pulled election results.

In [ ]:
# your optional code here
...

<br>

---

### Question 5
What if we want to use `groupby.agg` but aggregate using a different function for each column? One way to do this is by passing in a dictionary with the keys as column names and the values as function names to `.agg()`. 

Note: Shoutout to Connor for the question after lecture! 

#### Question 5a
Let's try it out. We are interested in learning a bit about our candidates' election experiences over the years. We want to know:

* the last year they ran,
* how many times they ran, and
* if they ever won.

Set `elections_multifn` to be a dataframe with this information, a result of calling `groupby.agg` using a dictionary of functions. `elections_multifn` should look similar to what is shown below. Small caveat: the `Party` column shown below may have a different column name.

<img src = "pics/elections_multifn.png" width = "300">

In [ ]:
fn_dict = ...
elections_multifn = ...
elections_multifn

In [ ]:
grader.check("q5a")

#### Question 5b

These column names are uninformative and incorrect. Rename the corresponding columns to be the following:
* `Most Recent Year`
* `Number of Elections`
* `Ever Won`

Hint: Remember you can use another dictionary here! 

In [ ]:
col_dict = ...
elections_multifn = ...
elections_multifn

In [ ]:
grader.check("q5b")

#### Question 5c

Now let's try to do it **without** using a dictionary of functions. First, create three dataframes, `elections_lastyr`, `elections_numtimes`, and `elections_everwon`, where each one is the result of applying the relevant function using `groupby.agg`. Then, merge the three of them using `pd.merge` twice. The final dataframe should be called `elections_multifn_manual`. 

Hint: You can either use `reset_index` to get `Candidate` back as a column or check out the `pd.merge` [documentation](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) to see how to merge on the index.

In [ ]:
elections_lastyr = ...
elections_numtimes = ...
elections_everwon = ...

elections_multifn_manual = ...
elections_multifn_manual = ...

elections_multifn_manual

In [ ]:
grader.check("q5c")

To finish, let's take a look at which candidates ran the most times...

In [ ]:
elections_multifn.sort_values('Number of Elections', ascending=False).head(10)

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Congratulations! You have finished Lab 03!

Mitski and Guppy congratulate you!

<img src = "pics/mitski.jpg" width = "300"> <img src = "pics/guppy.jpg" width = "400">

### Course Content Feedback

If you have any feedback about this assignment or about any of our other weekly, weekly assignments, lectures, or discussions, please fill out the [Course Content Feedback Form](https://forms.gle/owfPCGgnrju1xQEA9). Your input is valuable in helping us improve the quality and relevance of our content to better meet your needs and expectations!

### Submission Instructions

Below, you will see a cell. Running this cell will automatically generate a zip file with your autograded answers. Submit this file to the Lab 03 assignment on Gradescope. If you run into any issues when running this cell, feel free to check this [section](https://ds100.org/debugging-guide/autograder_gradescope/autograder_gradescope.html#why-does-grader.exportrun_teststrue-fail-if-all-previous-tests-passed) in the Data 100 Debugging Guide.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)